In [1]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress
from scipy.stats import sem

# Hide warning messages in notebook
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Declare where are the files to load, then read them as CSV and merge them (using the "Mouse ID")
mouse_data = pd.read_csv("data/mouse_drug_data.csv")
clinical_data = pd.read_csv("data/clinicaltrial_data.csv")
tutti_data = pd.merge(mouse_data, clinical_data, how="outer", on="Mouse ID" )
tutti_data.head()

,Mouse ID,Drug,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,f234,Stelasyn,0,45.000000,0
1,f234,Stelasyn,5,47.313491,0
2,f234,Stelasyn,10,47.904324,0
3,f234,Stelasyn,15,48.735197,1
4,f234,Stelasyn,20,51.112713,2


In [3]:
# General exploration of the data
#tutti_data.info()

In [4]:
# General exploration of the data
#tutti_data.describe()

In [5]:
# Second version, with Timepoint as "object" instead of integer
# Declare where are the files to load, then read them as CSV and merge them (using the "Mouse ID")
mouse_data = pd.read_csv("data/mouse_drug_data.csv")
clinical_data = pd.read_csv("data/clinicaltrial_data.csv")
tutti_data_v2 = pd.merge(mouse_data, clinical_data, how="outer", on="Mouse ID" )

# Adjusting Timepoints as objects, instead of integers (if needed)
tutti_data_v2["Timepoint"] = tutti_data_v2["Timepoint"].astype("object")
#tutti_data.dtypes
tutti_data_v2.head()

,Mouse ID,Drug,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,f234,Stelasyn,0,45.000000,0
1,f234,Stelasyn,5,47.313491,0
2,f234,Stelasyn,10,47.904324,0
3,f234,Stelasyn,15,48.735197,1
4,f234,Stelasyn,20,51.112713,2


In [6]:
# General exploration of the data
#tutti_data_v2.info()

In [7]:
# Store the Mean Tumor Volume Data Grouped by Drug and Timepoint 
g_by_d_and_t = tutti_data_v2.groupby(["Drug", "Timepoint"])
mtv = g_by_d_and_t["Tumor Volume (mm3)"].mean()
mms = g_by_d_and_t["Metastatic Sites"].mean()
n_mice = g_by_d_and_t["Mouse ID"].nunique()
mtv_se = g_by_d_and_t["Tumor Volume (mm3)"].sem()
mms_se = g_by_d_and_t["Metastatic Sites"].sem()

# Convert to DataFrames
mtv_by_drug_time_df = pd.DataFrame({"Mean Tumor Volume (mm3)" : mtv})
mms_by_drug_time_df = pd.DataFrame({"Mean Metastatic Sites" : mms})
nmice_by_drug_time_df = pd.DataFrame({"Mouse Count" : n_mice})

# Task 1. Tumor Response to Treatment
# Task 1.1 Preview DataFrame
#mtv_by_drug_time_df

# Task 1.2 Store the Standard Error of WHATEVER Grouped by Drug and Timepoint
se_mtv_by_drug_time_df = pd.DataFrame({"Tumor Volume Standard Error": mtv_se})
se_mms_by_drug_time_df = pd.DataFrame({"Metastatic Sites Standard Error" : mms_se})

# Task 1.2 Preview
#se_mtv_by_drug_time_df

# 1.3 Minor Data Munging to Re-Format the Data Frames
# Using Pivot_table, then rearange the columns
munging_mtv = pd.pivot_table(mtv_by_drug_time_df,values="Mean Tumor Volume (mm3)", index="Timepoint", columns="Drug")
munging_se_mtv = pd.pivot_table(se_mtv_by_drug_time_df,values="Tumor Volume Standard Error", index="Timepoint", columns="Drug")

# 1.3 Preview of reformatte worked
#munging_mtv.head()
munging_se_mtv.head()

Drug,Capomulin,Ceftamin,Infubinol,Ketapril,Naftisol,Placebo,Propriva,Ramicane,Stelasyn,Zoniferol
Timepoint,,,,,,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.448593,0.164505,0.235102,0.264819,0.202385,0.218091,0.231708,0.482955,0.239862,0.188950
10,0.702684,0.236144,0.282346,0.357421,0.319415,0.402064,0.376195,0.720225,0.433678,0.263949
15,0.838617,0.332053,0.357705,0.580268,0.444378,0.614461,0.466109,0.770432,0.493261,0.370544
20,0.909731,0.359482,0.476210,0.726484,0.595260,0.839609,0.555181,0.786199,0.621889,0.533182


In [ ]:
# 1.4 # Generate the Scatter Plot (with Error Bars)
#fig, ax = plt.subplots()

#ax.errorbar(munging_se_mtv, munging_mtv["Capomulin"], yerr=munging_se_mtv["Capomulin"], label="Mean of Tumor Volume")
#ax.set_xlim(0, len(munging_mtv)+1)
#ax.set_xlabel("Time")
#ax.set_ylabel("Mean Tumor Volume")
#plt.legend(loc="best", fontsize="small", fancybox=True)

x = np.arange(10)
y = 2.5 * np.sin(x / 20 * np.pi)
yerr = np.linspace(0.05, 0.2, 10)

plt.scatter(mtv_by_drug_time_df, munging_mtv, marker="o", facecolors="red", edgecolors="black")
plt.errorbar(x, y + 1, yerr=yerr, uplims=True, lolims=True, label="This option")
#fig, ax = plt.subplots()
#ax.errorbar(munging_se_mtv, munging_mtv, label="Mean of Tumor Volume")

# Show the Figure
plt.show()


#fig = plt.figure()
#x = np.arange(10)
#y = 2.5 * np.sin(x / 20 * np.pi)
#yerr = np.linspace(0.05, 0.2, 10)
#plt.errorbar(x, y + 3, yerr=yerr, label='both limits (default)')
#plt.errorbar(x, y + 2, yerr=yerr, uplims=True, label='uplims=True')
#plt.errorbar(x, y + 1, yerr=yerr, uplims=True, lolims=True, label="This option")
#upperlimits = [True, False] * 5
#lowerlimits = [False, True] * 5
#plt.errorbar(x, y, yerr=yerr, uplims=upperlimits, lolims=lowerlimits,label='subsets of uplims and lolims')




#plt.ylim(180,620)
#plt.xlim(11,26)
#plt.title("Ice Cream Sales v Temperature")
#plt.xlabel("Temperature (Celsius)")
#plt.ylabel("Sales (Dollars)")


# THIS KIND OF WORK, BUT NOT YET. 1.4 # Generate the Plot (with Error Bars)
#fig, ax = plt.subplots()
#ax.errorbar(np.arange(0, len(munging_se_mtv), 1)+1, munging_mtv["Capomulin"], yerr=munging_se_mtv["Capomulin"], label="Mean of Tumor Volume")
#ax.set_xlim(0, len(munging_mtv)+1)
#ax.set_xlabel("Time")
#ax.set_ylabel("Mean Tumor Volume")
#plt.legend(loc="best", fontsize="small", fancybox=True)
#plt.show()


# Save the Figure



In [ ]:
# Task 1.2 Store the Standard Error of WHATEVER Grouped by Drug and Timepoint
#se_mtv_by_drug_time_df

In [ ]:
# 1.3 Minor Data Munging to Re-Format the Data Frames
# Preview that Reformatting worked
# Using Pivot_table, then rearange the columns and print
#munging_mtv = pd.pivot_table(mtv_by_drug_time_df,values="Mean Tumor Volume (mm3)", index="Timepoint", columns="Drug")
#munging_se_mtv = pd.pivot_table(se_mtv_by_drug_time_df,values="Tumor Volume Standard Error", index="Timepoint", columns="Drug")
#munging_mtv.head()
#munging_se_mtv.head()

In [ ]:
# 1.4 # Generate the Plot (with Error Bars)
fig, ax = plt.subplots()
ax.errorbar(np.arange(0, len(mtv_by_drug_time_df), 1)+1, means, yerr=munging_se_mtv, fmt="o", color="r",
            alpha=0.5, label="Mean of Whatever")
ax.set_xlim(0, len(means)+1)
ax.set_xlabel("Time")
ax.set_ylabel("Mean Tumor Volume")
plt.legend(loc="best", fontsize="small", fancybox=True)
plt.show()

# Save the Figure

# Show the Figure

In [ ]:
# Task 2. Metastatic Response to Treatment
# Task 2.1 Preview DataFrame
mms_by_drug_time_df

In [ ]:
# Task 2.2 Store the Standard Error associated with Met. Sites Grouped by Drug and Timepoint 
# Convert to DataFrames
# Preview DataFrame
se_mms_by_drug_time_df

In [ ]:
# 2.3 Minor Data Munging to Re-Format the Data Frames
# Preview that Reformatting worked
# Using Pivot_table, then rearange the columns and print
munging_mms = pd.pivot_table(mms_by_drug_time_df,values="Mean Metastatic Sites", index="Timepoint", columns="Drug")
munging_mms.head()

In [ ]:
# Task 3. Survival Rates
# Task 3.1 Preview DataFrame
nmice_by_drug_time_df

In [ ]:
# This step is not needed for the Survival activity

# Convert to DataFrames

# Minor Data Munging to Re-Format the Data Frames
# Preview that Reformatting worked

# Preview DataFrame

In [ ]:
# Generate the Plot (with Error Bars)

# Save the Figure

# Show the Figure
#plt.show()

In [ ]:
# Make a variable called "drug_name" and store a "drug" in it
#drug_name = "SEA00001"
# (Capomulin, Ceftamin, Infubinol, Ketapril, Naftisol, Placebo, Propriva, Ramicane, Stelasyn, Zoniferol)

# Collect the trips of the 'bikeid' above
#just_capo = sum_it_up.loc[bike_id]

# Place the gender keys for that single bike into a list
#gender_list = just_capo.keys()

In [ ]:
# Task 1. Trying different options

# Set line
#(slope, intercept, _, _, _) = linregress(x_axis, fake)
#fit = slope * x_axis + intercept

# Plot data
#fig, ax = plt.subplots()

#fig.suptitle("Fake Banana Data!", fontsize=16, fontweight="bold")

#ax.set_xlim(0, 10)
#ax.set_ylim(0, 10)

#ax.set_xlabel("Fake Banana Ages (in days)")
#ax.set_ylabel("Fake Banana Weights (in Hundres of Kilograms)")

#ax.plot(x_axis, fake, linewidth=0, marker='o')
#ax.plot(x_axis, fit, 'b--')

#plt.show()

In [ ]:
# Task 1. Creating a scatter plot that shows how the tumor volume changes over time for each treatment.
    # Formatting: You must use proper labeling of your plots, including aspects like: 
    # Formatting: Plot Titles, Axes Labels, Legend Labels, X and Y Axis Limits, etc.
    # Formatting: Your scatter plots must include error bars.
   

In [ ]:
# Task 2. Creating a scatter plot that shows how the number of metastatic (cancer spreading) sites 
# changes over time for each treatment.
    # Formatting: You must use proper labeling of your plots, including aspects like: 
    # Formatting: Plot Titles, Axes Labels, Legend Labels, X and Y Axis Limits, etc.
    # Formatting: Your scatter plots must include error bars.

In [ ]:
# Task 3. Creating a scatter plot that shows the number of mice still alive 
# through the course of treatment (Survival Rate)
    # Formatting: You must use proper labeling of your plots, including aspects like: 
    # Formatting: Plot Titles, Axes Labels, Legend Labels, X and Y Axis Limits, etc.
    # Formatting: Your scatter plots must include error bars.


In [ ]:
# Task 4. Creating a bar graph that compares the total % tumor volume change for each drug across the full 45 days.
    # Formatting: Titles, Axes Labels, Legend Labels, etc.
    # Formatting: Your bar graph should indicate tumor growth as red and tumor reduction as green.
    # Formatting: It should also include a label with the percentage change for each bar. 


In [ ]:
# Task 5. Include 3 observations about the results of the study. Use the visualizations you generated 
# from the study data as the basis for your observations.

In [ ]:
# Generate groups by treatments 
# (Capomulin, Ceftamin, Infubinol, Ketapril, Naftisol, Placebo, Propriva, Ramicane, Stelasyn, Zoniferol)
 
# Using GroupBy in order to organize the data into fields according to "Drugs/Treatments"
#capo_df = tutti_data.groupby(["Drug"])
# In order to be visualized, a data function must be used
# The numeric variables can be now calculated per drug
#school_average_math = grouped_schools_df["math_score"].mean()
# Average Reading Score
#school_average_reading = grouped_schools_df["reading_score"].mean()
# Looking for the ones that pass math per school
#pass_math_per_school = pass_math_df.groupby(["school_name"])
# In order to get the percentage passing math
#perc_pass_math = (pass_math_per_school["Student ID"].count() / grouped_schools_df["Student ID"].count())*100
# Looking for the ones that pass reading per school
#pass_reading_per_school = pass_reading_df.groupby(["school_name"])
# In order to get the percentage passing reading
#perc_pass_reading = (pass_reading_per_school["Student ID"].count()/grouped_schools_df["Student ID"].count())*100
# Overall Passing Rate (Average of math and reading)
#perc_overall_passing = (perc_pass_math+perc_pass_reading)/2
# Create a school summary for future calculations (tasks 7 and onward), but a formatted version for tasks 2,3 and 4.
#school_summary = pd.DataFrame({"Total Students" : total_students, 
#                               "Total School Budget" : budget_per_school,
#                               "Per Student Budget" : budget_per_student,
#                               "Average Math Score" : school_average_math,
#                               "Average Reading Score" : school_average_reading,
#                               "% Passing Math" : perc_pass_math,
#                               "% Passing Reading" : perc_pass_reading,
#                               "Overall Passing Rate" : perc_overall_passing})
# Now, print to make sure all OK 
#school_summary_f